In [29]:
import numpy as np
import pandas as pd
import torch

In [30]:
data = pd.read_csv("Tweets.csv")
data = data.drop("textID",axis=1)
data.head()

,text,selected_text,sentiment
0,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,my boss is bullying me...,bullying me,negative
3,what interview! leave me alone,leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [31]:
from sklearn.preprocessing import LabelEncoder

In [32]:
le = LabelEncoder()
le_sentiment = le.fit_transform(data["sentiment"])
le_sentiment

array([1, 0, 0, ..., 2, 2, 1])

In [33]:
text_columns = ['text', 'selected_text']  # замени на реальные названия
data["combined_text"] = data[text_columns].apply(lambda x: ' '.join(x.astype(str)), axis=1)

In [34]:
X = data["combined_text"]
y = le_sentiment

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# TFIDF

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [39]:
from catboost import CatBoostClassifier

In [40]:
cb = CatBoostClassifier()

In [41]:
cb.fit(X_train_tfidf, y_train, verbose=100)

Learning rate set to 0.092634
0:	learn: 1.0789771	total: 138ms	remaining: 2m 17s
100:	learn: 0.8090495	total: 18.8s	remaining: 2m 47s
200:	learn: 0.7342771	total: 36s	remaining: 2m 23s
300:	learn: 0.6901654	total: 52.3s	remaining: 2m 1s
400:	learn: 0.6610516	total: 1m 8s	remaining: 1m 42s
500:	learn: 0.6402536	total: 1m 25s	remaining: 1m 25s
600:	learn: 0.6234167	total: 1m 41s	remaining: 1m 7s
700:	learn: 0.6105992	total: 1m 59s	remaining: 51s
800:	learn: 0.5999486	total: 2m 16s	remaining: 33.8s
900:	learn: 0.5917173	total: 2m 32s	remaining: 16.8s
999:	learn: 0.5843536	total: 2m 48s	remaining: 0us


In [42]:
tfidf_preds = cb.predict(X_test_tfidf)

In [43]:
from sklearn.metrics import classification_report


In [44]:
print(classification_report(y_test, tfidf_preds))

              precision    recall  f1-score   support

           0       0.83      0.61      0.70      1563
           1       0.66      0.86      0.74      2195
           2       0.87      0.74      0.80      1739

    accuracy                           0.75      5497
   macro avg       0.78      0.74      0.75      5497
weighted avg       0.77      0.75      0.75      5497



# Bert Tokenizer

In [45]:
text = "[CLS] Prepared by experienced English teachers, the texts, articles and conversations are brief and appropriate to your level of proficiency. [SEP] Take the multiple-choice quiz following each text, and you'll get the results immediately. [SEP] You will feel both challenged and accomplished! [SEP] You can even download (as PDF) and print the texts and exercises. [SEP] It's enjoyable, fun and free. [SEP] Good luck! [SEP]"

In [46]:
# Попробуем альтернативный импорт
from transformers import AutoTokenizer, AutoModel

# Используем auto-классы вместо конкретных Bert классов
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
model.eval()

print("✅ Модель успешно загружена!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

✅ Модель успешно загружена!


In [47]:
# КОРОЧЕ ТУТ ХУЙНЯ, ЧТО Я ПОПРОБОВАЛ ПРЕОБРАЗОВАТЬ ТЕКСТ В ТЕНЗОР, И ОН ПОЛУЧИЛСЯ СЛИШКОМ БОЛЬШИМ, А ТАК НОРМ ВРОДЕ
encoded = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=512,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

In [48]:
tokenized_text = tokenizer.tokenize(text)

print (tokenized_text)

['[CLS]', 'prepared', 'by', 'experienced', 'english', 'teachers', ',', 'the', 'texts', ',', 'articles', 'and', 'conversations', 'are', 'brief', 'and', 'appropriate', 'to', 'your', 'level', 'of', 'proficiency', '.', '[SEP]', 'take', 'the', 'multiple', '-', 'choice', 'quiz', 'following', 'each', 'text', ',', 'and', 'you', "'", 'll', 'get', 'the', 'results', 'immediately', '.', '[SEP]', 'you', 'will', 'feel', 'both', 'challenged', 'and', 'accomplished', '!', '[SEP]', 'you', 'can', 'even', 'download', '(', 'as', 'pdf', ')', 'and', 'print', 'the', 'texts', 'and', 'exercises', '.', '[SEP]', 'it', "'", 's', 'enjoyable', ',', 'fun', 'and', 'free', '.', '[SEP]', 'good', 'luck', '!', '[SEP]']


In [49]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
prepared      4,810
by            2,011
experienced   5,281
english       2,394
teachers      5,089
,             1,010
the           1,996
texts         6,981
,             1,010
articles      4,790
and           1,998
conversations 11,450
are           2,024
brief         4,766
and           1,998
appropriate   6,413
to            2,000
your          2,115
level         2,504
of            1,997
proficiency  26,293
.             1,012
[SEP]           102
take          2,202
the           1,996
multiple      3,674
-             1,011
choice        3,601
quiz         19,461
following     2,206
each          2,169
text          3,793
,             1,010
and           1,998
you           2,017
'             1,005
ll            2,222
get           2,131
the           1,996
results       3,463
immediately   3,202
.             1,012
[SEP]           102
you           2,017
will          2,097
feel          2,514
both          2,119
challenged    8,315
and           1,998

In [50]:
# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [51]:
# Собираем токены и segment IDs
all_tokens = ['[CLS]']
segments_ids = [0]  # [CLS] всегда относится к первому предложению

for i, sentence in enumerate(text):
    # Токенизируем предложение
    sentence_tokens = tokenizer.tokenize(sentence)
    all_tokens.extend(sentence_tokens)

    # Добавляем segment ID для этого предложения
    segments_ids.extend([i] * len(sentence_tokens))

    # Добавляем [SEP] после каждого предложения
    all_tokens.append('[SEP]')
    segments_ids.append(i)

# Конвертируем в индексы
indexed_tokens = tokenizer.convert_tokens_to_ids(all_tokens)
attention_mask = [1] * len(indexed_tokens)

print("Segment IDs:", segments_ids)

Segment IDs: [0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 15, 15, 16, 16, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 48, 48, 49, 49, 50, 50, 51, 52, 52, 53, 53, 54, 54, 55, 55, 56, 56, 57, 57, 58, 59, 59, 60, 60, 61, 61, 62, 62, 63, 63, 64, 64, 65, 65, 66, 66, 67, 68, 68, 69, 69, 70, 70, 71, 72, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77, 78, 78, 79, 79, 80, 80, 81, 81, 82, 82, 83, 83, 84, 84, 85, 86, 86, 87, 87, 88, 88, 89, 90, 90, 91, 91, 92, 92, 93, 93, 94, 94, 95, 96, 96, 97, 97, 98, 98, 99, 100, 100, 101, 101, 102, 102, 103, 103, 104, 104, 105, 105, 106, 106, 107, 107, 108, 108, 109, 109, 110, 110, 111, 112, 112, 113, 113, 114, 115, 115, 116, 116, 117, 117, 118, 118, 119, 120, 120, 121, 121, 122, 122, 123, 123, 124, 124, 125, 126, 126, 127, 127

In [52]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [53]:
model = AutoModel.from_pretrained(
    'bert-base-uncased',
    output_hidden_states=True  # Whether the model returns all hidden-states
)


# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()



BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [54]:

with torch.no_grad():
    outputs = model(
        input_ids=encoded['input_ids'],
        token_type_ids=encoded['token_type_ids'],
        attention_mask=encoded['attention_mask']
    )
    hidden_states = outputs.hidden_states

In [55]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))


Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 512
Number of hidden units: 768


In [56]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()


torch.Size([13, 1, 512, 768])

In [57]:
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 512, 768])

In [58]:
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([512, 13, 768])

In [59]:
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))


Shape is: 512 x 3072


In [60]:
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)

    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))


Shape is: 512 x 768


In [61]:
# `hidden_states` has shape [13 x 1 x 22 x 768]

# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

print ("Our final sentence embedding vector of shape:", sentence_embedding.size())


Our final sentence embedding vector of shape: torch.Size([768])


In [62]:
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)


0 [CLS]
1 prepared
2 by
3 experienced
4 english
5 teachers
6 ,
7 the
8 texts
9 ,
10 articles
11 and
12 conversations
13 are
14 brief
15 and
16 appropriate
17 to
18 your
19 level
20 of
21 proficiency
22 .
23 [SEP]
24 take
25 the
26 multiple
27 -
28 choice
29 quiz
30 following
31 each
32 text
33 ,
34 and
35 you
36 '
37 ll
38 get
39 the
40 results
41 immediately
42 .
43 [SEP]
44 you
45 will
46 feel
47 both
48 challenged
49 and
50 accomplished
51 !
52 [SEP]
53 you
54 can
55 even
56 download
57 (
58 as
59 pdf
60 )
61 and
62 print
63 the
64 texts
65 and
66 exercises
67 .
68 [SEP]
69 it
70 '
71 s
72 enjoyable
73 ,
74 fun
75 and
76 free
77 .
78 [SEP]
79 good
80 luck
81 !
82 [SEP]


In [63]:
print(str(token_vecs_sum[6][:5]))
print(str(token_vecs_sum[10][:5]))
print(str(token_vecs_sum[19][:5]))


tensor([ 2.9332,  0.0633,  0.3148, -1.4752, -0.0455])
tensor([ 2.9274,  4.9763,  2.9102,  1.4239, -0.2388])
tensor([-2.9008,  0.4244,  4.3595,  0.5140, -0.4609])


In [64]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd

# ===========================
# Настройки
# ===========================
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 32  # Можно уменьшить, если памяти мало

# ===========================
# Пример DataFrame
# ===========================
# Предположим, твоя колонка с текстами называется 'combined_text'
# data = pd.DataFrame({'combined_text': [...]})

# ===========================
# Загружаем BERT
# ===========================
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased', output_hidden_states=True).to(DEVICE)
model.eval()
print("✅ Модель успешно загружена!")

# ===========================
# Список для sentence embeddings
# ===========================
all_sentence_embeddings = []

texts = data['combined_text'].tolist()
num_batches = (len(texts) + BATCH_SIZE - 1) // BATCH_SIZE

for i in range(num_batches):
    batch_texts = texts[i*BATCH_SIZE : (i+1)*BATCH_SIZE]

    # Токенизация батча
    encoded = tokenizer(
        batch_texts,
        add_special_tokens=True,
        max_length=512,
        padding=True,
        truncation=True,
        return_tensors='pt'
    ).to(DEVICE)

    # Прогон через модель
    with torch.no_grad():
        outputs = model(
            input_ids=encoded['input_ids'],
            token_type_ids=encoded['token_type_ids'],
            attention_mask=encoded['attention_mask']
        )
        hidden_states = outputs.hidden_states  # [13 x batch x seq_len x 768]

    # Усредняем по токенам второго к последнему слоя
    for b in range(len(batch_texts)):
        sentence_embedding = torch.mean(hidden_states[-2][b], dim=0)
        all_sentence_embeddings.append(sentence_embedding.cpu())  # сразу на CPU

    if i % 100 == 0:
      print(f"✅ Обработан батч {i+1}/{num_batches}")

# ===========================
# Добавляем в DataFrame
# ===========================
data['sentence_embedding'] = all_sentence_embeddings

print("✅ Все тексты векторизованы")
print(f"Обработано {len(data)} текстов")
data.head()

✅ Модель успешно загружена!
✅ Обработан батч 1/859
✅ Обработан батч 101/859
✅ Обработан батч 201/859
✅ Обработан батч 301/859
✅ Обработан батч 401/859
✅ Обработан батч 501/859
✅ Обработан батч 601/859
✅ Обработан батч 701/859
✅ Обработан батч 801/859
✅ Все тексты векторизованы
Обработано 27481 текстов


,text,selected_text,sentiment,combined_text,sentence_embedding
0,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going I`d have ...","[tensor(0.3407), tensor(-0.4012), tensor(0.436..."
1,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!...,"[tensor(0.0211), tensor(0.3991), tensor(1.0126..."
2,my boss is bullying me...,bullying me,negative,my boss is bullying me... bullying me,"[tensor(0.6384), tensor(0.3904), tensor(1.1164..."
3,what interview! leave me alone,leave me alone,negative,what interview! leave me alone leave me alone,"[tensor(-0.1100), tensor(0.0607), tensor(0.817..."
4,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on t...","[tensor(0.4579), tensor(0.2682), tensor(0.5473..."


In [65]:
data.to_csv("dsadas.csv", index=False)

# Training catboost on bert vec

In [66]:
data_vec = data.copy()

In [67]:
data_vec.head()

,text,selected_text,sentiment,combined_text,sentence_embedding
0,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going I`d have ...","[tensor(0.3407), tensor(-0.4012), tensor(0.436..."
1,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!...,"[tensor(0.0211), tensor(0.3991), tensor(1.0126..."
2,my boss is bullying me...,bullying me,negative,my boss is bullying me... bullying me,"[tensor(0.6384), tensor(0.3904), tensor(1.1164..."
3,what interview! leave me alone,leave me alone,negative,what interview! leave me alone leave me alone,"[tensor(-0.1100), tensor(0.0607), tensor(0.817..."
4,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on t...","[tensor(0.4579), tensor(0.2682), tensor(0.5473..."


In [68]:
data_vec.shape

(27481, 5)

In [69]:
data.shape

(27481, 5)

In [70]:
data_vec["sentence_embedding"]

,sentence_embedding
0,"[tensor(0.3407), tensor(-0.4012), tensor(0.436..."
1,"[tensor(0.0211), tensor(0.3991), tensor(1.0126..."
2,"[tensor(0.6384), tensor(0.3904), tensor(1.1164..."
3,"[tensor(-0.1100), tensor(0.0607), tensor(0.817..."
4,"[tensor(0.4579), tensor(0.2682), tensor(0.5473..."
...,...
27476,"[tensor(0.0811), tensor(-0.3869), tensor(0.770..."
27477,"[tensor(0.4745), tensor(-0.3253), tensor(0.402..."
27478,"[tensor(0.0561), tensor(0.1640), tensor(0.8672..."
27479,"[tensor(-0.3237), tensor(0.5986), tensor(1.248..."


In [71]:
model = CatBoostClassifier(verbose=100)

In [72]:
le = LabelEncoder()
le_sentiment = le.fit_transform(data["sentiment"])
le_sentiment

array([1, 0, 0, ..., 2, 2, 1])

In [73]:
X_train, X_test, y_train, y_test = train_test_split(data_vec["sentence_embedding"], le_sentiment, test_size=0.2, random_state=0)

In [74]:
# используй, если в колонке тензоры
# X_train_np = np.array([tensor.numpy() for tensor in X_train])
# X_test_np = np.array([tensor.numpy() for tensor in X_test])

In [75]:
# если все по ублюдски сохранилось в виде строк
# Функция для безопасного преобразования
import ast

def safe_tensor_conversion(obj):
    if isinstance(obj, torch.Tensor):
        return obj.numpy()
    elif isinstance(obj, str):
        try:
            # Обрабатываем строковое представление тензора
            clean_str = obj.replace('tensor(', '').replace(')', '')
            return np.array(ast.literal_eval(clean_str), dtype=np.float32)
        except:
            return np.array(ast.literal_eval(obj), dtype=np.float32)
    else:
        return np.array(obj, dtype=np.float32)

# Преобразуем данные
X_train_np = np.array([safe_tensor_conversion(x) for x in X_train])
X_test_np = np.array([safe_tensor_conversion(x) for x in X_test])

In [76]:
model.fit(X_train_np, y_train)

Learning rate set to 0.092634
0:	learn: 1.0470315	total: 148ms	remaining: 2m 27s
100:	learn: 0.4407781	total: 11.8s	remaining: 1m 44s
200:	learn: 0.3569379	total: 22.5s	remaining: 1m 29s
300:	learn: 0.3136003	total: 33.2s	remaining: 1m 17s
400:	learn: 0.2841621	total: 43.7s	remaining: 1m 5s
500:	learn: 0.2599516	total: 54s	remaining: 53.8s
600:	learn: 0.2388519	total: 1m 4s	remaining: 42.8s
700:	learn: 0.2218889	total: 1m 14s	remaining: 31.9s
800:	learn: 0.2074249	total: 1m 24s	remaining: 21.1s
900:	learn: 0.1940636	total: 1m 35s	remaining: 10.5s
999:	learn: 0.1820445	total: 1m 45s	remaining: 0us


In [77]:
bert_preds = model.predict(X_test_np)

In [78]:
print(classification_report(y_test, bert_preds))

              precision    recall  f1-score   support

           0       0.86      0.80      0.82      1563
           1       0.85      0.93      0.89      2195
           2       0.88      0.83      0.86      1739

    accuracy                           0.86      5497
   macro avg       0.86      0.85      0.86      5497
weighted avg       0.86      0.86      0.86      5497

